# Imports and Setup

In [2]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

2024-07-02 10:22:41.489696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 10:22:41.489793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 10:22:41.594539: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Preparation

In [3]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [4]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


# Model Definition - VGG19

In [5]:
# Deep Feature Extraction - VGG19
def model_vgg19():
    VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in VGG_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(VGG_model.output)
    output = Model(inputs=VGG_model.input, outputs=feature)
    return output

# Training and Evaluation - Random Forest

In [5]:
# Random Forest Classifier - VGG19 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_19, y_train)
        test_pred = rf.predict(test_feature_19)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 24s 107ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step
Run 1 - Accuracy: 0.96250000, Recall: 0.96250000, Precision: 1.00000000, F1 Score: 0.98074449, AUC: 0.98125000
161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step
Run 2 - Accuracy: 0.96375000, Recall: 0.96375000, Precision: 1.00000000, F1 Score: 0.98119703, AUC: 0.98187500
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
Run 3 - Accuracy: 0.95625000, Recall: 0.95625000, Precision: 1.00000000, F1 Score: 0.97721528, AUC: 0.97812500
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 4 - Accuracy: 0.95125000, Recall: 0.95125000, Precision: 1.00000000, F1 Score: 0.97450684, AUC: 0.97562500
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 5 - Accuracy: 0.96750000, Recall: 0.96750000, Precision: 1.00000000, F1 Score: 0.9832

# Training and Evaluation - KNN

In [6]:
# KNN Classifier - VGG19 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)

        knn = KNeighborsClassifier()
        knn = knn.fit(train_feature_19, y_train)
        test_pred = knn.predict(test_feature_19)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 1 - Accuracy: 0.98125000, Recall: 0.98125000, Precision: 0.98171381, F1 Score: 0.98116742, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 2 - Accuracy: 0.98125000, Recall: 0.98125000, Precision: 0.98171381, F1 Score: 0.98116742, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 3 - Accuracy: 0.98125000, Recall: 0.98125000, Precision: 0.98171381, F1 Score: 0.98116742, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 4 - Accuracy: 0.98125000, Recall: 0.98125000, Precision: 0.98171381, F1 Score: 0.98116742, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 5 - Accuracy: 0.98125000, Recall: 0.98125000, Precision: 0.98171381, F1 Score: 0.98116742, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Training and Evaluation - SVM

In [8]:
# SVM Classifier - VGG19 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        svm = SVC(probability=True)
        svm.fit(train_feature_19, y_train_cat)
        test_pred = svm.predict(test_feature_19)
        test_pred_proba = svm.predict_proba(test_feature_19)

        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 4 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 5 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Training and Evaluation - Decision Tree Classifier

In [6]:
from sklearn.tree import DecisionTreeClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(train_feature_19, y_train)
        test_pred = dt.predict(test_feature_19)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 25s 112ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 1 - Accuracy: 0.90875000, Recall: 0.90875000, Precision: 0.91119352, F1 Score: 0.90921881, AUC: 0.93916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 2 - Accuracy: 0.90625000, Recall: 0.90625000, Precision: 0.90819162, F1 Score: 0.90596707, AUC: 0.93750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
Run 3 - Accuracy: 0.90750000, Recall: 0.90750000, Precision: 0.90821638, F1 Score: 0.90727216, AUC: 0.93833333
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 4 - Accuracy: 0.91750000, Recall: 0.91750000, Precision: 0.91932756, F1 Score: 0.91704064, AUC: 0.94500000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 5 - Accuracy: 0.90750000, Recall: 0.90750000, Precision: 0.90835677, F1 Score: 0.9070

# Training and Evaluation - XGBoost Classifier

In [8]:
from xgboost import XGBClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train XGBoost Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(train_feature_19, y_train_cat)
        test_pred = xgb.predict(test_feature_19)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, xgb.predict_proba(test_feature_19), multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step
Run 1 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 0.99997083
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
Run 2 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 0.99997083
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 3 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 0.99997083
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step
Run 4 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 0.99997083
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 5 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 0.99997083
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Results and Metrics